# Models comparison for sentence similarity

In [37]:
from sentence_transformers import SentenceTransformer, util

from itertools import product, combinations

## ES

In [44]:
data = """
¿Y que hombro y qué arte,
podrían retorcer la nervadura de tu corazón
Y cuando tu corazón comenzó a latir
¿Qué formidable mano, qué formidables pies?

¿Y qué hombro, y qué arte
pudo tejer la nervadura de tu corazón?
Y al comenzar los latidos de tu corazón,
¿qué mano terrible? ¿Qué terribles pies?
"""
texts = data.split('\n')
texts = [_ for _ in texts if _]
texts

['¿Y que hombro y qué arte,',
 'podrían retorcer la nervadura de tu corazón',
 'Y cuando tu corazón comenzó a latir',
 '¿Qué formidable mano, qué formidables pies?',
 '¿Y qué hombro, y qué arte',
 'pudo tejer la nervadura de tu corazón?',
 'Y al comenzar los latidos de tu corazón,',
 '¿qué mano terrible? ¿Qué terribles pies?']

In [47]:
comb = list(combinations(texts, 2))

In [129]:
# https://huggingface.co/models?pipeline_tag=sentence-similarity&language=es&sort=likes
test_models = [
    "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli",
    "clips/mfaq",
    "hiiamsid/sentence_similarity_spanish_es",
    "hackathon-pln-es/bertin-roberta-base-finetuning-esnli",
]

In [130]:
def comparison_matrix(embeddings):
    length = len(embeddings)
    matrix = np.zeros((length, length))

    for i in range(length):
        for j in range(length):
            if i < j:  # ignore diagonal and lower-left redundant side
                pair = embeddings[i], embeddings[j]
                metric = util.cos_sim(*pair).numpy().squeeze()
                metric = metric.item(0)  # scalar
                matrix[i, j] = metric
                # print(metric, pair)

    return matrix

In [106]:
embeddings = [sentence_model.encode(_) for _ in texts]

8

In [125]:
threshold = 0.5

In [135]:
for model in test_models:
    print('\nMODEL: ', model)    
    sentence_model = SentenceTransformer(model)
    embeddings = [sentence_model.encode(_) for _ in texts]

    matrix = comparison_matrix(embeddings)
    boolean_matrix = (matrix >= threshold).astype(int)
    # zero all values below threshold
    matrix[matrix < threshold] = 0
    print(boolean_matrix)
    
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if matrix[i, j] > threshold:
                print('\t', i, j, texts[i][:25] + '...', '\t', texts[j][:25] + '...')



MODEL:  symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli
[[0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
	 0 4 ¿Y que hombro y qué arte,... 	 ¿Y qué hombro, y qué arte...
	 1 5 podrían retorcer la nerva... 	 pudo tejer la nervadura d...
	 1 6 podrían retorcer la nerva... 	 Y al comenzar los latidos...
	 2 6 Y cuando tu corazón comen... 	 Y al comenzar los latidos...
	 5 6 pudo tejer la nervadura d... 	 Y al comenzar los latidos...

MODEL:  clips/mfaq
[[0 0 0 1 1 0 0 1]
 [0 0 1 1 0 1 1 1]
 [0 0 0 1 0 1 1 1]
 [0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0]]
	 0 3 ¿Y que hombro y qué arte,... 	 ¿Qué formidable mano, qué...
	 0 4 ¿Y que hombro y qué arte,... 	 ¿Y qué hombro, y qué arte...
	 0 7 ¿Y que hombro y qué arte,... 	 ¿qué mano terrible? ¿Qué ...
	 1 2 podrían retorcer la nerva... 	 Y cuando tu corazón comen...
	 1 3 podrían retorcer 